## 1. Carga de librerias


In [8]:
! python -m pip install --upgrade pip
! pip install keras
! pip install tensorflow

Requirement already up-to-date: pip in c:\users\william.hincapie\appdata\local\continuum\anaconda3\lib\site-packages (19.0.2)
  Stored in directory: C:\Users\william.hincapie\AppData\Local\pip\Cache\wheels\5c\2e\7e\a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
  Stored in directory: C:\Users\william.hincapie\AppData\Local\pip\Cache\wheels\7c\06\54\bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Stored in directory: C:\Users\william.hincapie\AppData\Local\pip\Cache\wheels\90\db\f8\2c3101f72ef1ad434e4662853174126ce30201a3e163dcbeca
Successfully built gast termcolor absl-py


In [9]:
### General
import numpy as np
import matplotlib as plt
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_validate
import math

### Error
## ECM
from sklearn.metrics import mean_squared_error
## Coeficiente de determinacion
from sklearn.metrics import r2_score
## Varianza explicada
from sklearn.metrics import explained_variance_score

### Regresión polinomial múltiple
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

### Ventana de Parzen
from scipy.spatial import distance

### Redes Neuronales Artificiales
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

### Random Forest
from sklearn.ensemble import RandomForestRegressor

### Vectores de soporte
from sklearn import svm

Using TensorFlow backend.


## 2. Carga de la base de datos

In [4]:
data = pd.read_csv("data/bank.csv", delimiter=";",header='infer')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
